This script implements the simulated studies for conducting $\mbox{QReR}_{\rm M}$$(p_a=0.1)$ to estimate $\tau_{PATE}$.

In [1]:
import os
import itertools

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd
from tqdm import tqdm

from datagen import *
from network import *
from benchmarks import *

from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler

In [3]:
def expand_grid(data_dict):
    """Create a dataframe from every combination of given values."""
    rows = itertools.product(*data_dict.values())
    return pd.DataFrame.from_records(rows, columns=data_dict.keys())

# Perform Simulation

In [4]:
save_folder_root = './save/0415qrer_pate/'

In [5]:
def parallel_unit(i,r,batch_size,
                  lr,pa,
                  num_nodes,num_iters,
                  num_init_iters,
                  x_lambda,
                  wt_lambda,
                  kernel_params,val_metric,
                  patience,random_state,
                  data_path):
  
    print('------------- Data:',i,'------------- ')

    # load the data
    data_full_path = data_path + 'd' + str(i) + '.npy'
    dat = np.load(data_full_path,allow_pickle=True)

    # unzip the data
    x = dat.item()['x']
    z = dat.item()['z']
    y1 = dat.item()['y1']
    y2 = dat.item()['y2']
    y3 = dat.item()['y3']
        
    nt = int(z.sum())
    x = x[:nt*(1+r)]
    z = z[:nt*(1+r)]
    y1 = y1[:nt*(1+r)]
    y2 = y2[:nt*(1+r)]
    y3 = y3[:nt*(1+r)]
    
    sc = StandardScaler()
    x = sc.fit_transform(x)
    
    ######## Perform QReR ########
    save_folder = save_folder_root+scenario+'/'+'r='+str(r)+'/pa='+str(pa)+'/'+str(i)+'/'

    if not os.path.exists(save_folder):
        print('Creat the folder.')
        os.makedirs(save_folder)

    if not os.path.exists(save_folder+'final_checkpoint.pt'):
        print('Train the model from scratch.')

        estimator = QReR(lr=lr,
                          batch_size=batch_size,
                          patience=patience,
                          num_iters=num_iters,
                          num_init_iters=num_init_iters,
                          pa=pa,
                          x_lambda=x_lambda,
                          wt_lambda=wt_lambda,
                          num_nodes=num_nodes,
                          val_metric=val_metric,
                          save_folder=save_folder,
                          kernel_params=kernel_params,
                          verbose=False,
                          random_state=random_state)

        # train the model from scratch
        estimator.fit(x,z)

    else:
        print('Skip! The model has been trained.')
        estimator = QReR(lr=lr,
                          batch_size=batch_size,
                          patience=patience,
                          num_iters=num_iters,
                          num_init_iters=1,
                          pa=pa,
                          x_lambda=x_lambda,
                          wt_lambda=wt_lambda,
                          num_nodes=num_nodes,
                          val_metric=val_metric,
                          save_folder=save_folder,
                          kernel_params=kernel_params,
                          verbose=False,
                          random_state=random_state)
        estimator.w = z
        estimator.nwts = int(estimator.w.shape[0])
        estimator.nt = int(z.sum())
        estimator.nc = int((1-z).sum())
        estimator._init_network()
        estimator.netG.load_state_dict(torch.load(save_folder+'final_checkpoint.pt'))
    
    ######## Conduct the Estimation ########
    np.random.seed(i)
    torch.manual_seed(i)
    
    wts_mat_net = estimator.predict(1000).numpy()
    
    if not os.path.exists(save_folder+'qrer_est.csv'):
        # QReR-M
        avg_wts = wts_mat_net.mean(axis=0)

        est1_m = tau_diff(y1,z,avg_wts)
        est2_m = tau_diff(y2,z,avg_wts)
        est3_m = tau_diff(y3,z,avg_wts)

        df_est = pd.DataFrame({
          'tauhat': [est1_m,est2_m,est3_m],
          'type': 'M'
        })
        
        df_est.to_csv(save_folder+"qrer_est.csv",index=False)
    else:
        print('Skip! QReR has been considered')
        
    return pd.read_csv(save_folder+"qrer_est.csv").values

In [6]:
def mcse_cal_fun(array_val,tau):
    # array value
    n_sim = array_val.shape[0]

    # bias MCSE
    bias_mcse = np.sqrt(np.var(array_val)/n_sim)

    # RMSE MCSE using asymptotic property
    # using Delta method and the formula in 
    # https://cran.r-project.org/web/packages/simhelpers/vignettes/MCSE.html
    array_sq_val = (array_val-tau)**2
    mse = array_sq_val.mean()
    rmse_mcse = np.sqrt(np.sum((array_sq_val-mse)**2)/((n_sim-1)*4*mse*n_sim))

    # # RMSE MCSE using bootstrapping
    # from sklearn.utils import resample
    # B = 1000 # bootstrapping iteration
    # rmse_boot_array = []
    # for b in range(B):
    #     array_val_boot = resample(array_val,
    #                               n_samples=n_sim,
    #                               replace=1)
    #     rmse_boot_array.append(np.sqrt(((array_val_boot-tau)**2).mean()))
    # rmse_boot_array = np.array(rmse_boot_array)
    # rmse_mcse = np.sqrt(np.var(rmse_boot_array))

    return bias_mcse, rmse_mcse

In [7]:
n_kernel = 40
n_data = 200
tau = 1

scenarios = ['scenario1','scenario2','scenario3']
rs = [1,2]

result1_df = []
result2_df = []
result3_df = []

In [ ]:
for scenario in scenarios:
    for r in rs:
        
        print('----------------',scenario,'r =',r,'----------------')
        
        data_path = './save/simu_data/'+scenario+'/'

        net_params = {
         'r':[r],
         'data_path': [data_path],
            'batch_size': [512],
         'lr': [0.001],
         'pa': [0.1],
         'val_metric': ['KS'],
         'num_nodes': [512],
         'num_iters': [5000],
         'num_init_iters': [500],
         'patience': [15],
         'x_lambda': [1],
         'wt_lambda': [1],
         'kernel_params': [{'kernel':'rbf',
                         'gamma':10,
                         'degree':2,
                         'c':0}],
         'random_state': [0]}
        
        param_df = expand_grid(net_params)

        for i_param in range(param_df.shape[0]):
            kwargs = dict(param_df.iloc[i_param,:])
            print('----------------- [%d/%d] -----------------\n'%(i_param+1,param_df.shape[0]))
            results_all = Parallel(n_jobs=n_kernel,verbose=0)(delayed(parallel_unit)(i=i,**kwargs) for i in tqdm(range(n_data)))

            # QReR
            dat_array = np.array([results_all[i] for i in range(n_data)])
            bias = dat_array[:,:,0].mean(axis=0)-tau
            rmse = np.sqrt(np.mean((dat_array[:,:,0]-tau)**2,axis=0).astype(float))
            # covarage = ((dat_array[:,:,1]<=tau)*(dat_array[:,:,2]>=tau)).mean(axis=0)
            # width = (dat_array[:,:,2] - dat_array[:,:,1]).mean(axis=0)
            mcse = np.apply_along_axis(func1d=mcse_cal_fun,
                                        arr=dat_array[:,:,0],
                                        axis=0,tau=tau)
            bias_mcse = mcse[0,:]
            rmse_mcse = mcse[1,:]

            result_dict = {'Bias':bias,
                           'Bias_MCSE':bias_mcse,
                           'RMSE':rmse,
                           'RMSE_MCSE':rmse_mcse,
                           #'CI Covarage':covarage,
                           #'CI Width':width,
                           'pa':param_df.iloc[i_param,4],
                           'Method':['QReR-M' for i in range(dat_array.shape[1])],
                           'Outcome': ['Linear','Nonlinear1','Nonlinear2'],
                           'r':r,
                           'Scenario':scenario}

            result_df = pd.DataFrame(result_dict)
            result1_df.append(result_df.iloc[0,:])
            result2_df.append(result_df.iloc[1,:])
            result3_df.append(result_df.iloc[2,:])

---------------- scenario1 r = 1 ----------------
----------------- [1/1] -----------------



100%|██████████| 1/1 [00:00<00:00, 23.53it/s]  | 40/200 [00:00<00:02, 57.57it/s]

 40%|████████████████▊                         | 80/200 [00:03<00:05, 22.25it/s]

------------- Data: 37 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
------------- Data: 17 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
------------- Data: 5 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
------------- Data: 39 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
------------- Data: 35 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
------------- Data: 22 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
------------- Data: 16 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
------------- Data: 32 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.


  0%|          | 0/1 [00:00<?, ?it/s]

------------- Data: 30 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretraining complete!
Skip! QReR has been considered
------------- Data: 41 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretraining complete!
Skip! QReR has been considered
------------- Data: 40 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
------------- Data: 11 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
------------- Data: 8 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretraining complete!
Skip! QReR has been considered
------------- Data: 42 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretraining complete!
Skip! QReR has been considered
------------- Data: 43 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretr

...

 40%|████████████████▍                        | 80/200 [00:00<00:00, 417.21it/s]

Pretraining complete!
Skip! QReR has been considered
------------- Data: 0 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretraining complete!
Skip! QReR has been considered
------------- Data: 3 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretraining complete!
Skip! QReR has been considered
------------- Data: 1 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretraining complete!
Skip! QReR has been considered
------------- Data: 2 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretraining complete!
Skip! QReR has been considered
------------- Data: 7 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretraining complete!
Skip! QReR has been considered
------------- Data: 5 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretraining comp

100%|████████████████████████████████████████| 200/200 [00:00<00:00, 395.84it/s]

Pretraining complete!
Skip! QReR has been considered
------------- Data: 40 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretraining complete!
Skip! QReR has been considered
------------- Data: 43 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretraining complete!
Skip! QReR has been considered
------------- Data: 44 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretraining complete!
Skip! QReR has been considered
------------- Data: 45 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretraining complete!
Skip! QReR has been considered
------------- Data: 48 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretraining complete!
Skip! QReR has been considered
------------- Data: 47 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretrainin

Pretraining complete!
Skip! QReR has been considered
------------- Data: 72 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretraining complete!
Skip! QReR has been considered
------------- Data: 73 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretraining complete!
Skip! QReR has been considered
------------- Data: 70 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretraining complete!
Skip! QReR has been considered
------------- Data: 74 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretraining complete!
Skip! QReR has been considered
------------- Data: 75 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretraining complete!
Skip! QReR has been considered
------------- Data: 76 ------------- 
Skip! The model has been trained.
Initialize the network via pretraining.
Pretrainin

# Generate Tables

In [ ]:
pd.DataFrame(result1_df).set_index(['r',
                                    'Scenario',
                                    'Outcome'])

In [ ]:
pd.DataFrame(result2_df).set_index(['r',
                                    'Scenario',
                                    'Outcome'])

In [ ]:
pd.DataFrame(result3_df).set_index(['r',
                                    'Scenario',
                                    'Outcome'])

In [ ]:
result_df = pd.DataFrame(result1_df+result2_df+result3_df).set_index(['r','Scenario','Outcome'])
result_df.to_csv(save_folder_root+'qrer_pate.csv')
result_df